In [9]:
from random import randint, choice, shuffle, random
from pprint import pprint
from itertools import permutations
from math import inf as oo # Infinity (∞) is larger than any number
from math import sqrt
from time      import time
import matplotlib.pyplot as plt
import copy
import numpy as np

MAX_DISTANCE = 100

def random_symmetric_graph(n):
    ''' Symmetric adjacency matrix of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i+1,n):
            v = randint(1,MAX_DISTANCE)
            dist_matrix[i][j] = v
            dist_matrix[j][i] = v
    return dist_matrix

def random_euclidean_graph(n):
    ''' Symmetric adjacency matrix of a Euclidean graph of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    points = []
    for p in range(n):
        x,y = randint(0,MAX_DISTANCE), randint(0,MAX_DISTANCE)
        points.append((x,y))
    for i in range(n):
        p1 = points[i]
        for j in range(i+1,n):
            p2 = points[j]
            distance = sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
            dist_matrix[i][j] = distance
            dist_matrix[j][i] = distance
    return dist_matrix

def show(G):
    ''' Show adjacency matrix. Useful for debugging. '''
    n = len(G)
    r = "     "
    for i in range(n):
        r += f'{i:4}'
    r += '\n    -'+'-'*(4*n)+'\n'
    for i in range(n):
        r += f'{i:2} | '
        for j in range(n):
            r += f'{G[i][j]:4}'
        r += '\n'
    r = r.replace('inf', '  ∞')
    print(r)

def cost(G, cycle):
    ''' Calculate the cost of the given cycle '''
    c = 0
    n = len(G)
    for i in range(n):
        a = cycle[i]
        b = cycle[(i+1)%n]
        c += G[a][b]
    return c

# Example

# Traveling Salesman Problem using Genetic Algorithm

Source:
* https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35
*  https://www.geeksforgeeks.org/traveling-salesman-problem-using-genetic-algorithm/ (in C++)
* http://www.theprojectspot.com/tutorial-post/applying-a-genetic-algorithm-to-the-travelling-salesman-problem/5
* https://www.youtube.com/watch?v=XP8R0yzAbdo

# Create a first random generation of x possible routes

In [10]:
G = random_symmetric_graph(5)
population = 6

print(G)

show(G)

'''Used the shuffle function to generate a 'population' sized amount'''
'''of possible routes'''

def createGen(matrix, population):
    route = []
    genZ = [None] * int(population)
    for i in range(0,len(matrix)):
        route.append(i)
    for j in range(0,int(population)):
        if j == 0:
            np.random.shuffle(route)
            newRoute = route[:]
            genZ[j] = newRoute
        else:
            np.random.shuffle(route)
            for h in range(0,len(genZ)):
                if route == genZ[h]:
                    np.random.shuffle(route)
            newRoute = route[:]
            genZ[j] = newRoute
    return genZ

print("Genenration 0")        
print(createGen(G,6))

[[inf, 47, 56, 62, 87], [47, inf, 5, 100, 68], [56, 5, inf, 17, 9], [62, 100, 17, inf, 70], [87, 68, 9, 70, inf]]
        0   1   2   3   4
    ---------------------
 0 |    ∞  47  56  62  87
 1 |   47   ∞   5 100  68
 2 |   56   5   ∞  17   9
 3 |   62 100  17   ∞  70
 4 |   87  68   9  70   ∞

Genenration 0
[[4, 0, 3, 1, 2], [3, 4, 1, 2, 0], [0, 4, 1, 2, 3], [3, 4, 1, 0, 2], [0, 2, 3, 1, 4], [3, 1, 0, 4, 2]]


# Rank the "fitness" of these routes aka length

In [11]:
'''Turn routes into fitness values'''

def fitness(matrix, routes):
    fitnessRanking = []
    for i in range(0, len(routes)):
        fitnessTotal = 0
        for j in range(0,len(matrix)):
            if j + 1 < len(matrix):
                fitnessTotal = fitnessTotal + int(matrix[routes[i][j]][routes[i][j+1]])
            else:
                fitnessTotal = fitnessTotal + int(matrix[routes[i][j]][routes[i][0]])
        fitnessRanking.append(fitnessTotal)
    return fitnessRanking

show(G)
print("Genenration 0")        
genZ = createGen(G,6)
print(genZ)
print("Fitness")
ranking = fitness(G, genZ)
print(ranking)

        0   1   2   3   4
    ---------------------
 0 |    ∞  47  56  62  87
 1 |   47   ∞   5 100  68
 2 |   56   5   ∞  17   9
 3 |   62 100  17   ∞  70
 4 |   87  68   9  70   ∞

Genenration 0
[[0, 1, 4, 2, 3], [1, 2, 0, 3, 4], [1, 3, 4, 0, 2], [4, 3, 2, 1, 0], [1, 4, 0, 2, 3], [1, 0, 3, 2, 4]]
Fitness
[203, 261, 318, 226, 328, 203]


# Choose the parents for mating - Elitism

Find two routes with the lowest value, shortest route

In [12]:
def first(ranking, routes):
    sortedRank = ranking[:]
    sortedRank.sort()
    firstPos = ranking.index(sortedRank[0])
    firstRoute = routes[firstPos]
    return firstRoute

def second(ranking, routes):
    sortedRank = ranking[:]
    sortedRank.sort()
    secondPos = ranking.index(sortedRank[1])
    secondRoute = routes[secondPos]
    return secondRoute
    
win = first(ranking, genZ)
print(win)
win2 = second(ranking, genZ)
print(win2)
    

[0, 1, 4, 2, 3]
[0, 1, 4, 2, 3]


# Breading

In [13]:
def breed(parent1, parent2, population):
    nextGen = []
    
    for i in range(0,population):
        child = []
        childP1 = []
        childP2 = []
    
    
        geneA = int(np.random.random() * len(parent1))
        geneB = int(np.random.random() * len(parent1))
    
        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)
    
        for i in range(startGene, endGene):
            childP1.append(parent1[i])
        
        childP2 = [item for item in parent2 if item not in childP1]
        child = childP1 + childP2
        nextGen.append(child)
    
    return nextGen
    
nextGen = breed(win, win2, population)
print(nextGen)

[[1, 0, 4, 2, 3], [1, 0, 4, 2, 3], [0, 1, 4, 2, 3], [0, 1, 4, 2, 3], [1, 0, 4, 2, 3], [1, 4, 2, 0, 3]]


# Mutation

Randomly mutate with a chosen rate

In [16]:
'''Mutation rate set to 5%'''

mutation = 0.05

def mutate(child, mutationRate):
    for swapped in range(len(child)):
        if(np.random.random() <= mutationRate):
            swapWith = int(np.random.random() * len(child))
            
            node1 = child[swapped]
            node2 = child[swapWith]
            
            child[swapped] = node2
            child[swapWith] = node1
    return child

def mutateGen(currentGen, mutationRate, population):
    mutatedGen = []
    
    for child in range(0, population):
        mutatedChild = mutate(currentGen[child], mutationRate)
        mutatedGen.append(mutatedChild)
        
    return mutatedGen

mutatedGen = mutateGen(nextGen,mutation, population)
print(mutatedGen)

[[1, 0, 4, 2, 3], [1, 0, 4, 2, 3], [0, 1, 4, 2, 3], [0, 1, 4, 2, 3], [1, 0, 4, 2, 3], [1, 4, 2, 0, 3]]


# Repeat

Put all functions into one to produce new generation

In [17]:

def nextGeneration(mutatedGen, population, mutationRate):
    rank = fitness(G, mutatedGen)
    parent1 = first(rank, mutatedGen)
    parent2 = second(rank, mutatedGen)
    currentGen = breed(parent1, parent2, population)
    nextGeneration = mutateGen(currentGen, mutationRate, population)
    return nextGeneration

print(nextGeneration(mutatedGen, population, mutation))

[[2, 0, 1, 4, 3], [4, 2, 0, 1, 3], [1, 0, 4, 2, 3], [0, 3, 4, 2, 1], [0, 1, 4, 2, 3], [1, 0, 4, 2, 3]]


# Evolution

In [33]:
def geneticAlgorithm(population, matrix, mutationRate, generations):
    gen = createGen(matrix, population)
    rank = fitness(G, gen)
    rank.sort()
    print("Initial distance: ")
    print(rank[0])
    
    for i in range(0, int(generations)):
        gen = nextGeneration(gen, population, mutationRate)
    
    rank = fitness(G, gen)
    rank.sort()
    print("Initial distance: ")
    print(rank[0])
    bestRoute = first(rank, gen)
    return bestRoute

bestRoute = geneticAlgorithm(population, G, mutation, 10)
print(bestRoute)

Initial distance: 
193
Initial distance: 
193
[3, 2, 1, 0, 4]


# Test

In [37]:
print("MAX_DISTANCE")
MAX_DISTANCE = 100
print(MAX_DISTANCE)

G = random_symmetric_graph(10)
print("\nMatrix")
show(G)

print("\nPopulation")
population = 10
print(population)

print("\nMutation in %")
mutation=0.05
print(mutation)

print("\nAmount of generations created")
generations = 10
print(generations)

print("\nRunning")
bestRoute = geneticAlgorithm(population, G, mutation, generations)
print("\n Best Route")
print(bestRoute)

MAX_DISTANCE
100

Matrix
        0   1   2   3   4   5   6   7   8   9
    -----------------------------------------
 0 |    ∞  11  60  29  30  94  66  75  15  78
 1 |   11   ∞ 100  69  28  15  70  96  11  16
 2 |   60 100   ∞  15  98  68  52  34  35  97
 3 |   29  69  15   ∞  53  68  12  51  31  57
 4 |   30  28  98  53   ∞  26  40  53  35  22
 5 |   94  15  68  68  26   ∞  70  94  11  29
 6 |   66  70  52  12  40  70   ∞  49  25  82
 7 |   75  96  34  51  53  94  49   ∞  39  73
 8 |   15  11  35  31  35  11  25  39   ∞  55
 9 |   78  16  97  57  22  29  82  73  55   ∞


Population
10

Mutation in %
0.05

Amount of generations created
10

Running
Initial distance: 
437
Initial distance: 
259

 Best Route
[8, 0, 4, 5, 1, 3, 2, 7, 6, 9]
